In [81]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.stats import norm

%run ../COMMON/common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

from source_data.src import DBSource, RowTypes
import source_data.prepare as prep

iFirstFactYear=2008
iLastFactYear=2019

iFirstForecastYear=iLastFactYear+1
iLastForecastYear=2030

In [82]:
%store -r

In [83]:
class bal_zap:
    pdfWork=None

    lstSvod=['squareddy_x','squareddy_ipot_x','squareddy_ss_x'] # результаты из блока "Цены и себестоимость"
    
    def __init__(self):
        assert False, 'you can\'t create variables of DemandOM class!'
    
    def MakeWorkFrame():
        ''' Загрузка исходных данных
            Исходные данные:
                Фактические значения - из базы данных year.sqlite3;
                Экзогенные - из базы данных exog_year.sqlite3
                Задаваемые вручную - из базы данных exog_param.sqlite3
                Результаты других моделей - из базы данных svod.sqlite3. 
        '''
        
        srcSvod  = DBSource(common.strSvodDBPath, RowTypes.MODEL, bal_zap.lstSvod)
        
        bal_zap.pdfWork=srcSvod.make_frame().combine_first(dfBZ)
        
        return bal_zap.pdfWork

In [84]:
bal_zap.MakeWorkFrame()

,BldProcMKD,ProjectsPrivate_x,VvodyMKD_private,squareddy_ipot_x,squareddy_ss_x,squareddy_x
2004,59.386684,27.254825,22.167600,NaN,NaN,NaN
2005,66.863359,30.726575,23.249900,NaN,NaN,NaN
2006,70.858939,31.297480,27.301900,NaN,NaN,NaN
2007,68.941974,29.248635,31.165600,NaN,NaN,NaN
2008,64.672504,29.003730,33.273200,3.741290,NaN,NaN
2009,68.476586,31.282002,27.477920,0.772200,NaN,NaN
2010,73.837691,34.126605,28.765500,1.705002,7.060626,8.765628
2011,81.098644,38.266952,31.006000,4.206192,9.382939,13.589131
2012,90.318934,43.570800,34.350510,7.147313,11.554604,18.701917
2013,99.986909,46.467375,36.799400,12.523024,8.939837,21.462861


In [85]:
bal_zap.pdfWork['sold_stock_in']=bal_zap.pdfWork['squareddy_x']
bal_zap.pdfWork['unsold_stock_in']=bal_zap.pdfWork['ProjectsPrivate_x']-bal_zap.pdfWork['sold_stock_in']

bal_zap.pdfWork.loc[2005:2009,'sold_stock']=0.65*bal_zap.pdfWork['BldProcMKD']
bal_zap.pdfWork.loc[2005:2009,'unsold_stock']=bal_zap.pdfWork['BldProcMKD']-bal_zap.pdfWork['sold_stock']

bal_zap.pdfWork.loc[2006:2010,'sold_stock_out']=pd.concat([bal_zap.pdfWork['sold_stock'].shift(1), bal_zap.pdfWork['VvodyMKD_private']*0.65], axis=1).min(axis=1)

for i in range (2010,iLastFactYear+1):
    bal_zap.pdfWork.loc[i,'sold_stock_out']=min(bal_zap.pdfWork.loc[i-1,'sold_stock'],
                                                bal_zap.pdfWork.loc[i,'VvodyMKD_private']*0.65)
    
    bal_zap.pdfWork.loc[i,'sold_stock']=(bal_zap.pdfWork.loc[i-1,'sold_stock']+
                                         bal_zap.pdfWork.loc[i,'sold_stock_in']-
                                         bal_zap.pdfWork.loc[i,'sold_stock_out'])
    
bal_zap.pdfWork.loc[2005:2019,'unsold_stock_out']=(bal_zap.pdfWork['VvodyMKD_private']-
                                                   bal_zap.pdfWork['sold_stock_out'])



In [86]:
for i in range (2010,iLastFactYear+1):
    if i<2017:
        bal_zap.pdfWork.loc[i,'unsold_stock']=(bal_zap.pdfWork.loc[i-1,'unsold_stock']+
                                               bal_zap.pdfWork.loc[i,'unsold_stock_in']-
                                               bal_zap.pdfWork.loc[i,'unsold_stock_out'])
    else:
        bal_zap.pdfWork.loc[i,'unsold_stock']=(bal_zap.pdfWork.loc[i,'BldProcMKD']-
                                               bal_zap.pdfWork.loc[i,'sold_stock'])

In [87]:
bal_zap.pdfWork['D14']=0
bal_zap.pdfWork.loc[2014,'D14']=1

pdfBZM = bal_zap.pdfWork.loc[iFirstFactYear:iLastFactYear,['unsold_stock_out','unsold_stock','D14']].dropna()
resBZM = smf.ols(formula='unsold_stock_out ~ unsold_stock.shift(1) + D14 -1', missing='drop', 
                    data=pdfBZM.loc[2009:2017]).fit()
print(resBZM.summary())

                                 OLS Regression Results                                
Dep. Variable:       unsold_stock_out   R-squared (uncentered):                   0.984
Model:                            OLS   Adj. R-squared (uncentered):              0.978
Method:                 Least Squares   F-statistic:                              182.0
Date:                Mon, 22 Mar 2021   Prob (F-statistic):                    4.26e-06
Time:                        20:29:14   Log-Likelihood:                         -16.813
No. Observations:                   8   AIC:                                      37.63
Df Residuals:                       6   BIC:                                      37.79
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

In [88]:
for i in range (iFirstForecastYear,iLastForecastYear+1):
    bal_zap.pdfWork.loc[i,'unsold_stock_out']=resBZM.predict(bal_zap.pdfWork).loc[i]
    bal_zap.pdfWork.loc[i,'unsold_stock']=(bal_zap.pdfWork.loc[i-1,'unsold_stock']+
                                           bal_zap.pdfWork.loc[i,'unsold_stock_in']-
                                           bal_zap.pdfWork.loc[i,'unsold_stock_out'])
    
    bal_zap.pdfWork.loc[i,'sold_stock_out']=(bal_zap.pdfWork.loc[i,'VvodyMKD_private']-
                                             bal_zap.pdfWork.loc[i,'unsold_stock_out'])
    bal_zap.pdfWork.loc[i,'sold_stock']=(bal_zap.pdfWork.loc[i-1,'sold_stock']+
                                         bal_zap.pdfWork.loc[i,'sold_stock_in']-
                                         bal_zap.pdfWork.loc[i,'sold_stock_out'])

In [90]:
bal_zap.pdfWork[['unsold_stock_out']]

,unsold_stock_out
2004,NaN
2005,NaN
2006,9.555665
2007,10.907960
2008,11.645620
2009,9.617272
2010,10.067925
2011,10.852100
2012,12.022678
2013,12.879790
